# Analysis Plan: Do Teams with Higher Average Player Age Perform Worse?

## Research Question
Investigate whether teams with higher average player age show worse performance in tournaments using PCA and statistical analysis.

---

## Detailed Analysis Plan

### Step 1: Data Loading & Initial Exploration
- Import libraries (pandas, numpy, sklearn, matplotlib, seaborn, scipy)
- Load player statistics CSV from FBref scraping
- Examine structure with `.head()`, `.info()`, `.describe()`
- Check for missing values and data types
- Identify available columns: Player, Team, Position, Age, Goals, Assists, Minutes, Matches Played, Games Started, xG, npxG, xAG, Progressive Carries, Progressive Passes, Cards, League, Country

### Step 2: Data Cleaning
- Remove rows with missing critical values (Age, Team)
- Convert numeric columns from strings to appropriate types
- Handle missing values in performance metrics
- Validate data quality before analysis

### Step 3: Age Parsing
- Parse age format "YY-DDD" (years-days) to extract years only
- Create new `Age_Years` column with integer values
- Validate parsing with descriptive statistics
- Visualize age distribution by league

### Step 4: Team-Level Performance Aggregation
- **Group players by Team and League**
- Calculate team average age (mean across all players)
- Aggregate performance metrics per team:
  - Total goals, assists, xG, npxG, xAG
  - Total progressive carries and passes
  - Total yellow/red cards
  - Squad size and average matches per player
- Create normalized per-match metrics for fair comparison:
  - Goals per match, Assists per match
  - xG per match, xAG per match
  - Progressive actions per match

### Step 5: Feature Standardization
- Select 6-8 key performance metrics for PCA
- Apply `StandardScaler` to ensure mean=0, std=1
- Prevents variables with larger scales from dominating PCA
- Validate scaling worked correctly

### Step 6: PCA Application (Core Technique)
- Fit PCA to standardized performance metrics
- Create **scree plot** to visualize variance explained by each component
- Determine optimal number of components (typically 2-3 for 80-95% variance)
- Analyze **component loadings** to interpret what each PC represents:
  - PC1 might represent "overall attacking output"
  - PC2 might represent "playing style" (direct vs progressive)
- Transform team data into principal component scores
- Visualize teams in PC1-PC2 space, colored by average age

### Step 7: Correlation Analysis (Age vs Performance)
- **Primary research objective:** Test relationship between team age and performance
- Calculate Pearson correlation (linear relationships)
- Calculate Spearman correlation (monotonic relationships)
- Test correlations for:
  - Age vs individual metrics (goals, xG, assists, etc.)
  - Age vs principal component scores (PC1, PC2)
- Extract p-values for significance testing (α = 0.05)
- Create summary table showing all correlations with significance markers
- Visualize scatter plots with regression lines for key metrics

### Step 8: Age Group Comparison
- Categorize teams into age groups:
  - Young teams (< 25 years average)
  - Middle-aged teams (25-27 years)
  - Older teams (> 27 years)
- Compare performance distributions using box plots
- Run statistical tests:
  - **ANOVA** (if normally distributed)
  - **Kruskal-Wallis** (non-parametric alternative)
- Perform post-hoc pairwise comparisons to identify which groups differ
- Test differences in PC scores across age groups

### Step 9: K-Means Clustering (Optional Advanced Analysis)
- Apply clustering on PCA-transformed data (2D space)
- Use **elbow method** to find optimal number of clusters (k=2-5)
- Calculate **silhouette scores** to validate cluster quality
- Analyze average age within each performance cluster
- Test if age differs significantly across clusters using ANOVA
- Interpretation: Do high-performing teams tend to be younger/older?

### Step 10: Comprehensive Visualizations
Create 6-8 key visualizations:
1. **Scree plot** - variance explained by each PC
2. **PCA biplot** - teams in PC1-PC2 space, colored by age
3. **Component loadings** - which metrics contribute to each PC
4. **Age distribution** - histogram and league comparison
5. **Correlation heatmap** - age vs all performance metrics
6. **Scatter plots** - age vs key metrics (goals, xG) with regression lines
7. **Box plots** - age group performance comparison
8. **Cluster visualization** - if clustering applied

### Step 11: Statistical Summary & Conclusions
- Synthesize all findings into clear statements
- Answer research question with statistical evidence:
  - "Teams with higher average age show **[positive/negative/no]** correlation with performance"
  - Report correlation coefficients and p-values
  - Identify which specific metrics are affected
- Discuss **limitations**:
  - Observational data (no causation)
  - Confounding factors (league quality, budget, injuries)
  - Missing variables (tactics, coach experience)
- Provide data-driven conclusions with confidence levels
- Suggest future research directions

---

## Available Performance Metrics
Based on FBref standard stats table:
- **Attacking:** Goals, Assists, xG (Expected Goals), npxG (non-penalty xG), xAG (Expected Assists)
- **Playing Style:** Progressive Carries, Progressive Passes
- **Discipline:** Yellow Cards, Red Cards
- **Participation:** Minutes, Matches Played, Games Started

**Note:** Shooting stats (Shots, Shots on Target) and passing completion are NOT available in this dataset and would require separate scraping from FBref shooting/passing pages.

## Step 1: Data Loading and Initial Exploration

In [200]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy import stats
import warnings
warnings.filterwarnings('ignore')



In [201]:
# Load the player statistics data
df = pd.read_csv('../data_raw/playerStats.csv')


In [202]:
""""Player": get_stat("player"),
            "Squad": get_stat("team"),
            "Pos": get_stat("position"),
            "Age": get_stat("age"),
            "Gls": get_stat("goals"),
            "Ast": get_stat("assists"),
            "Min": get_stat("minutes"),
            "MP": get_stat("games"),
            "GS": get_stat("games_starts"),
            "xG": get_stat("xg"),
            "npxG": get_stat("npxg"),
            "xAG": get_stat("xg_assist"),
            "PrgC": get_stat("progressive_carries"),
            "PrgP": get_stat("progressive_passes"),
            "Yellow": get_stat("cards_yellow"),
            "Red": get_stat("cards_red"),
            "League": league_config["league"],
            "Country": league_config["country"],
            "Source": source,
            "Source_URL": build_stats_url(league_config),"""
rows = [
    "Player",
    "Team",
    "Position",
    "Age",
    "Goals",
    "Assists",
    "Minutes",
    "Matches Played",
    "Games Started",
    "Expected Goals",
    "npxG",
    "Expected Assists",
    "PrgC",
    "PrgP",
    "Yellow",
    "Red",
    "League",
    "Country",
    "Source",
    "Source_URL",
]
df.columns = rows

In [203]:

print("Dataset Shape:", df.shape)

df.head()

Dataset Shape: (2361, 20)


,Player,Team,Position,Age,Goals,Assists,Minutes,Matches Played,Games Started,Expected Goals,npxG,Expected Assists,PrgC,PrgP,Yellow,Red,League,Country,Source,Source_URL
0,Brenden Aaronson,Leeds United,"FW,MF",25-064,1,2,911,16,10,2.1,2.1,1.5,21,26,1,0,Premier League,England,cloudscraper,https://fbref.com/en/comps/9/stats/Premier-Lea...
1,Zach Abbott,Nott'ham Forest,DF,19-226,0,0,45,1,0,0.0,0.0,0.0,0,1,0,0,Premier League,England,cloudscraper,https://fbref.com/en/comps/9/stats/Premier-Lea...
2,Joshua Acheampong,Chelsea,DF,19-234,1,0,349,5,4,0.6,0.6,0.0,4,11,0,0,Premier League,England,cloudscraper,https://fbref.com/en/comps/9/stats/Premier-Lea...
3,Tyler Adams,Bournemouth,MF,26-314,2,1,"1,195",15,15,0.5,0.5,0.1,12,61,5,0,Premier League,England,cloudscraper,https://fbref.com/en/comps/9/stats/Premier-Lea...
4,Tosin Adarabioyo,Chelsea,DF,28-092,0,0,561,9,5,0.1,0.1,0.0,3,18,1,0,Premier League,England,cloudscraper,https://fbref.com/en/comps/9/stats/Premier-Lea...


In [204]:

df.info()

df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2361 entries, 0 to 2360
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Player            2361 non-null   object 
 1   Team              2361 non-null   object 
 2   Position          2361 non-null   object 
 3   Age               2360 non-null   object 
 4   Goals             2361 non-null   int64  
 5   Assists           2361 non-null   int64  
 6   Minutes           2361 non-null   object 
 7   Matches Played    2361 non-null   int64  
 8   Games Started     2361 non-null   int64  
 9   Expected Goals    2361 non-null   float64
 10  npxG              2361 non-null   float64
 11  Expected Assists  2361 non-null   float64
 12  PrgC              2361 non-null   int64  
 13  PrgP              2361 non-null   int64  
 14  Yellow            2361 non-null   int64  
 15  Red               2361 non-null   int64  
 16  League            2361 non-null   object 


,Goals,Assists,Matches Played,Games Started,Expected Goals,npxG,Expected Assists,PrgC,PrgP,Yellow,Red
count,2361.000000,2361.000000,2361.000000,2361.000000,2361.000000,2361.000000,2361.000000,2361.000000,2361.000000,2361.000000,2361.000000
mean,0.872935,0.605676,10.225328,7.230834,0.908217,0.820627,0.632740,10.598899,23.865735,1.310462,0.070309
std,1.609279,1.046763,4.926069,5.364557,1.400194,1.193159,0.845632,12.659611,24.546691,1.442607,0.267069
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,6.000000,2.000000,0.100000,0.100000,0.000000,2.000000,4.000000,0.000000,0.000000
50%,0.000000,0.000000,11.000000,7.000000,0.400000,0.400000,0.300000,7.000000,17.000000,1.000000,0.000000
75%,1.000000,1.000000,14.000000,12.000000,1.200000,1.100000,0.900000,15.000000,36.000000,2.000000,0.000000
max,19.000000,8.000000,18.000000,18.000000,16.400000,14.800000,7.600000,116.000000,150.000000,9.000000,2.000000


In [205]:
# Check for missing values
print("Missing Values Summary:")
print("="*50)
print(f"Total number of null values in dataset: {df.isnull().sum().sum()}")
print("\nNull values per column:")
print(df.isnull().sum())

Missing Values Summary:
Total number of null values in dataset: 1

Null values per column:
Player              0
Team                0
Position            0
Age                 1
Goals               0
Assists             0
Minutes             0
Matches Played      0
Games Started       0
Expected Goals      0
npxG                0
Expected Assists    0
PrgC                0
PrgP                0
Yellow              0
Red                 0
League              0
Country             0
Source              0
Source_URL          0
dtype: int64


In [206]:
df = df[~df['Age'].isnull()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2360 entries, 0 to 2360
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Player            2360 non-null   object 
 1   Team              2360 non-null   object 
 2   Position          2360 non-null   object 
 3   Age               2360 non-null   object 
 4   Goals             2360 non-null   int64  
 5   Assists           2360 non-null   int64  
 6   Minutes           2360 non-null   object 
 7   Matches Played    2360 non-null   int64  
 8   Games Started     2360 non-null   int64  
 9   Expected Goals    2360 non-null   float64
 10  npxG              2360 non-null   float64
 11  Expected Assists  2360 non-null   float64
 12  PrgC              2360 non-null   int64  
 13  PrgP              2360 non-null   int64  
 14  Yellow            2360 non-null   int64  
 15  Red               2360 non-null   int64  
 16  League            2360 non-null   object 
 17  

## Step 3: Age Parsing
Parse the Age column to extract numeric age values

In [207]:
# Inspect the Age column format

print(df['Age'].head(10))
print(f"\nAge column data type: {df['Age'].dtype}")
print(f"Unique Age format examples: {df['Age'].unique()[:5]}")

0    25-064
1    19-226
2    19-234
3    26-314
4    28-092
5    23-358
6    25-229
7    28-191
8    29-270
9    29-078
Name: Age, dtype: object

Age column data type: object
Unique Age format examples: ['25-064' '19-226' '19-234' '26-314' '28-092']


In [208]:
# "YY-DDD" (e.g., "25-064")
# Split by '-' and take the first part (years)
df['Age_Years'] = df['Age'].astype(str).str.split('-').str[0].astype(int)

# Verify the parsing
print(f"Original Age format: {df['Age'].iloc[0]}")
print(f"Parsed Age (years): {df['Age_Years'].iloc[0]}")
print(f"\nAge_Years statistics:")
print(df['Age_Years'].describe())
print(f"\nAge range: {df['Age_Years'].min()} - {df['Age_Years'].max()} years")

Original Age format: 25-064
Parsed Age (years): 25

Age_Years statistics:
count    2360.000000
mean       25.688136
std         4.468365
min        15.000000
25%        22.000000
50%        25.000000
75%        29.000000
max        42.000000
Name: Age_Years, dtype: float64

Age range: 15 - 42 years


## Step 4: Team-Level Performance Aggregation
Aggregate player statistics to team level, considering position and minutes played

In [209]:
# First, prepare numeric columns for aggregation
# Convert performance metrics to numeric values
numeric_cols = ['Goals', 'Assists', 'Minutes', 'Matches Played', 'Games Started', 
                'Expected Goals', 'npxG', 'Expected Assists', 'PrgC', 'PrgP', 
                'Yellow', 'Red']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Check conversion
print("Data types after conversion:")
print(df[numeric_cols].dtypes)
print(f"\nMissing values after conversion:")
print(df[numeric_cols].isnull().sum())

Data types after conversion:
Goals                 int64
Assists               int64
Minutes             float64
Matches Played        int64
Games Started         int64
Expected Goals      float64
npxG                float64
Expected Assists    float64
PrgC                  int64
PrgP                  int64
Yellow                int64
Red                   int64
dtype: object

Missing values after conversion:
Goals                 0
Assists               0
Minutes             623
Matches Played        0
Games Started         0
Expected Goals        0
npxG                  0
Expected Assists      0
PrgC                  0
PrgP                  0
Yellow                0
Red                   0
dtype: int64


In [ ]:
df["Minutes"] = df["Minutes"].isnull().fillna(df["Minutes"].mean())
print(df.isnull().sum())
print(df.shape)

Player              0
Team                0
Position            0
Age                 0
Goals               0
Assists             0
Minutes             0
Matches Played      0
Games Started       0
Expected Goals      0
npxG                0
Expected Assists    0
PrgC                0
PrgP                0
Yellow              0
Red                 0
League              0
Country             0
Source              0
Source_URL          0
Age_Years           0
dtype: int64
(2360, 21)


In [215]:
# Create team-level aggregations
# Group by Team and League to calculate team statistics
team_stats = df.groupby(['Team', 'League']).agg({
    'Age_Years': 'mean',  # Average team age
    'Goals': 'sum',  # Total team goals
    'Assists': 'sum',  # Total team assists
    'Expected Goals': 'sum',  # Total team xG
    'npxG': 'sum',  # Total non-penalty xG
    'Expected Assists': 'sum',  # Total team xAG
    'PrgC': 'sum',  # Total progressive carries
    'PrgP': 'sum',  # Total progressive passes
    'Minutes': 'sum',  # Total minutes played
    'Matches Played': 'mean',  # Average matches per player
    'Yellow': 'sum',  # Total yellow cards
    'Red': 'sum',  # Total red cards
    'Player': 'count'  # Number of players in squad
}).reset_index()

# Rename columns for clarity
team_stats.columns = ['Team', 'League', 'Avg_Age', 'Total_Goals', 'Total_Assists', 
                       'Total_xG', 'Total_npxG', 'Total_xAG', 'Total_PrgC', 
                       'Total_PrgP', 'Total_Minutes', 'Avg_Matches_Per_Player',
                       'Total_Yellow', 'Total_Red', 'Squad_Size']

print(f"Team-level dataset shape: {team_stats.shape}")
team_stats.head(10)

Team-level dataset shape: (96, 15)


,Team,League,Avg_Age,Total_Goals,Total_Assists,Total_xG,Total_npxG,Total_xAG,Total_PrgC,Total_PrgP,Total_Minutes,Avg_Matches_Per_Player,Total_Yellow,Total_Red,Squad_Size
0,Alavés,La Liga,26.458333,13,6,18.3,15.2,11.6,221,540,5,11.208333,41,1,24
1,Angers,Ligue 1,25.040000,16,7,16.9,15.3,11.4,257,423,7,10.080000,22,1,25
2,Arsenal,Premier League,25.083333,29,21,29.6,27.2,18.5,358,767,7,10.833333,22,0,24
3,Aston Villa,Premier League,27.217391,26,20,18.8,18.8,14.9,246,595,8,11.565217,25,1,23
4,Atalanta,Serie A,25.739130,20,13,22.4,21.7,15.7,350,755,4,10.869565,24,1,23
5,Athletic Club,La Liga,26.400000,15,7,24.5,21.3,15.9,313,770,6,11.440000,37,3,25
6,Atlético Madrid,La Liga,27.347826,30,19,30.3,28.0,22.7,352,785,7,12.478261,33,2,23
7,Augsburg,Bundesliga,25.739130,17,11,17.1,16.3,12.3,203,522,5,10.173913,45,1,23
8,Auxerre,Ligue 1,24.040000,12,7,19.9,16.1,12.4,223,491,7,9.680000,37,6,25
9,Barcelona,La Liga,23.920000,50,37,42.2,37.4,29.9,445,976,9,11.320000,28,2,25


In [217]:
print(team_stats.shape)

(96, 15)


In [212]:
# Create per-match performance metrics (normalized by squad activity)
# This gives a better comparison across teams
# Select performance features for PCA
performance_features = ['Goals_Per_Match', 'Assists_Per_Match', 'xG_Per_Match', 
                        'xAG_Per_Match', 'PrgC_Per_Match', 'PrgP_Per_Match',
                        'Total_Yellow', 'Total_Red']


team_stats['Goals_Per_Match'] = team_stats['Total_Goals'] / team_stats['Avg_Matches_Per_Player']
team_stats['Assists_Per_Match'] = team_stats['Total_Assists'] / team_stats['Avg_Matches_Per_Player']
team_stats['xG_Per_Match'] = team_stats['Total_xG'] / team_stats['Avg_Matches_Per_Player']
team_stats['xAG_Per_Match'] = team_stats['Total_xAG'] / team_stats['Avg_Matches_Per_Player']
team_stats['PrgC_Per_Match'] = team_stats['Total_PrgC'] / team_stats['Avg_Matches_Per_Player']
team_stats['PrgP_Per_Match'] = team_stats['Total_PrgP'] / team_stats['Avg_Matches_Per_Player']

print("Performance metrics per match:")
print(team_stats[['Team', 'Avg_Age', 'Goals_Per_Match', 'xG_Per_Match', 'xAG_Per_Match']].head(10))

Performance metrics per match:
              Team    Avg_Age  Goals_Per_Match  xG_Per_Match  xAG_Per_Match
0           Alavés  26.458333         1.159851      1.632714       1.034944
1           Angers  25.040000         1.587302      1.676587       1.130952
2          Arsenal  25.083333         2.676923      2.732308       1.707692
3      Aston Villa  27.217391         2.248120      1.625564       1.288346
4         Atalanta  25.739130         1.840000      2.060800       1.444400
5    Athletic Club  26.400000         1.311189      2.141608       1.389860
6  Atlético Madrid  27.347826         2.404181      2.428223       1.819164
7         Augsburg  25.739130         1.670940      1.680769       1.208974
8          Auxerre  24.040000         1.239669      2.055785       1.280992
9        Barcelona  23.920000         4.416961      3.727915       2.641343


## Step 5: PCA on Performance Metrics
Apply PCA to identify the main dimensions of team performance